## Fuzzers && sanitizers

<br />

### Intro.

В ГОСТ Р 56939-2024 "Защита информации. Разработка безопасного программного обеспечения. Общие требования" фаззинг-тестирование и динамический анализ кода санитайзерами - обязательные требования к разработке ПО (чтобы оно было признано безопасным).  
https://base.garant.ru/410749342/

Фаззеры и санитайзеры - runtime инструменты поиска ошибок в программе. Это разные методы поиска ошибок, но наибольший эффект дают в совокупности.

(!) Фаззеры и санитайзеры должны быть частью регулярного CI-процесса наравне с юнит-тестированием.

<br />

### Sanitizers.

Санитайзеры - инструменты поиска ошибок в программах в процессе их исполнения.

Варианты санитайзеров:
* [asan: AddressSanitizer](https://clang.llvm.org/docs/AddressSanitizer.html) - детектор ошибок работы с памятью:
  * out-of-bounds access
  * use-after-free
  * use-after-return
  * use-after-scope
  * double free, invalid free
  * memory leaks
  * ...
* [msan: MemorySanitizer](https://clang.llvm.org/docs/MemorySanitizer.html) - детектор ошибок работы с памятью:
  * Uninitialized value was used in a conditional branch.
  * Uninitialized pointer was used for memory accesses.
  * Uninitialized value was passed or returned from a function call.
  * Uninitialized data was passed into some libc calls.
  * ...
* [ubsan: UndefinedBehaviourSanitizer](https://clang.llvm.org/docs/UndefinedBehaviorSanitizer.html) - детектор ошибок типа UB:
  * Array subscript out of bounds, where the bounds can be statically determined
  * Bitwise shifts that are out of bounds for their data type
  * Dereferencing misaligned or null pointers
  * Signed integer overflow
  * Conversion to, from, or between floating-point types which would overflow the destination
  * ...
* [tsan: ThreadSanitizer](https://clang.llvm.org/docs/ThreadSanitizer.html) - детектор ошибок типа data race.
* [lsan: LeakSanitizer](https://clang.llvm.org/docs/LeakSanitizer.html) - детектор утчечек памяти.
  * Интегрирован в asan, является его (отключаемой) частью и в норме не требует отдельного запуска.

<br />

#### Принцип работы санитайзеров.

Санитайзеры поддержаны компилятором. Для этого компилятор выполняет __инструментирование__ программ, т.е. добавляет код детекции и обработки ошибок в программу.

Рассмотрим простенький пример как ubsan может отловить разыменование нулевого указателя.

Пример кода программы:

```c++
// test.cpp

void copy_name(Person *dst, Person *src) {
    dst->name = src->name;
}
```

Если скомпилировать код с особым ключом `-fsanitize=undefined`:

```
clang++ -O3 -c test.cpp -fsanitize=undefined
```

То во время компиляции код будет дополнен проверками (примерно):

```c++
// test.cpp
void copy_name(Person *dst, Person *src) {
    if (!src) {
        __ubsan_handle_null_ptr_deref();
    }
    if (!dst) {
        __ubsan_handle_null_ptr_deref();
    }
    dst->name = src->name;
}
```

И затем уже скомпилирован.

<br />

**Замечание**: Имя `__ubsan_handle_null_ptr_deref` и набор параметров (пустой) выбраны для понятности примера. В реальности они будут другими.

**Замечание**: Реализация функции `__ubsan_handle_null_ptr_deref` - ответственность компилятора. Нужные функции поставляются в библиотеке вместе с компилятором и будут прилинкованы к исполняемому файлу при сборке.

**Замечание**: Закинуть код на godbolt.org и показать его ассемблер с `-fsanitize=address` и без него:

```c++
void foo(int *p) {
    *p = 5;
}
```

<br />

**Упражнение:** Подумайте, как можно было бы реализовать следующие динамические проверки:

* ubsan:
  * out-of-bounds access (compile-time bounds)
  * Bitwise shifts that are out of bounds for their data type
  * Signed integer overflow
* msan:
  * Uninitialized value was used in a conditional branch.
  * Uninitialized pointer was used for memory accesses.
* asan:
  * out-of-bounds access (runtime bounds)
  * use-after-free
  * double free, invalid free
* tsan:
  * data race.

<br />

#### Стоимость санитайзеров

Как можно догадаться из реализации, введение дополнительных проверок замедляет программу и может требовать больше памяти.

Из документации clang:
* ubsan: -
* asan: замедление 2х
* msan: замедление 3х, потребление памяти 2х - 3х
* tsan: замедление 5х - 15х, потребление памяти 5х - 10х

<br />

#### Примеры использования санитайзеров:

<br />

##### ubsan

Пример:

```c++
#include <cstdint>
#include <iostream>
#include <string>

int main(int argc, char **argv) {
  for (int i = 1; i < argc; ++i) {
    std::int64_t base1 = 1, base2 = 2;
    std::int64_t shift = std::stoll(argv[i]);
    std::cout << (base1 << shift) << ' '
              << (base2 << shift) << std::endl;
  }
  return 0;
}
```

* Если `shift >= 63`, то ub в выражении `base2 << shift` (выход за пределы знакового)
* Если `shift >= 64`, то ub в выражениях `base1 << shift` и `base2 << shift` (сдвиг >= размера типа)

Компилируем без динамических проверок, запустим:

```sh
$ clang++-18 main.cpp -O3
$ ./a.out 5 10 63 64 65
32 64
1024 2048
-9223372036854775808 0
1 2
2 4
```

Программа "отработала".

Компилируем с инструментированием ubsan, запустим:

```
$ clang++-18 main.cpp -O3 -fsanitize=undefined
$ ./a.out 5 10 63 64 65
32 64
1024 2048
main.cpp:10:25: runtime error: left shift of 2 by 63 places cannot be represented in type 'std::int64_t' (aka 'long')
SUMMARY: UndefinedBehaviorSanitizer: undefined-behavior main.cpp:10:25 
-9223372036854775808 0
main.cpp:9:25: runtime error: shift exponent 64 is too large for 64-bit type 'std::int64_t' (aka 'long')
SUMMARY: UndefinedBehaviorSanitizer: undefined-behavior main.cpp:9:25 
1 2
2 4
```

Т.е.:
  * При детектировании ошибки печатается, что произошло и где.
  * При детектировании ошибки выполнение продолжается (настраиваемо)
  * Ошибка может случиться несколько раз, но для каждого места печатается единожды.

**Замечание:** "По-хорошему", когда встретили UB в программе, её нужно сразу убивать, т.к. она уже представляет из себя зомби. Но режим с продолжением исполнения полезен, чтобы за один прогон собрать побольше данных для анализа.

<br />

##### asan

Пример:

```c++
#include <iostream>
#include <vector>

int main(int argc, char **argv) {
  std::vector<int> v = {1, 2, 3, 4};
  int &x = v[1];

  for (int i = 0; i < 16; ++i)
    v.push_back(i);

  std::cout << x << std::endl;
  return 0;
}
```

Где здесь ошибка?

Компилируем без динамических проверок, запустим:

```sh
$ clang++-18 main.cpp -O3
$ ./a.out
6
```

Программа "отработала".

Компилируем с инструментированием asan, запустим (приведён сокращённый вывод):

```
$ clang++-18 main.cpp -O3 -fsanitize=address
$ ./a.out
================================================================
==8902==ERROR: AddressSanitizer: heap-use-after-free on address 0x502000000014 at pc 0x5e5ba73b2ec8 bp 0x7ffeefbf9df0 sp 0x7ffeefbf9de8
READ of size 4 at 0x502000000014 thread T0
    #0 0x5e5ba73b2ec7 in main
    #1 0x79fa21a2a1c9 in __libc_start_call_main
    #2 0x79fa21a2a28a in __libc_start_main
    #3 0x5e5ba72d73b4 in _start

0x502000000014 is located 4 bytes inside of 16-byte region [0x502000000010,0x502000000020)
freed by thread T0 here:
    #0 0x5e5ba73b10a1 in operator delete(void*)
    #1 0x5e5ba73b2d0c in main

previously allocated by thread T0 here:
    #0 0x5e5ba73b0821 in operator new(unsigned long)
    #1 0x5e5ba73b2bb7 in main
    #2 0x79fa21a2a28a in __libc_start_main
    #3 0x5e5ba72d73b4 in _start

SUMMARY: AddressSanitizer: heap-use-after-free in main
==8902==ABORTING
```

<br />


##### msan

Пример:

```c++
#include <iostream>

struct Point {
  float x;
  float y;
};

Point *create_point() {
  return new Point;
}

int main(int argc, char **argv) {
  Point *p = create_point();

  if (p->x < 0.f)
    std::cout << "x is negative\n";
  else if (p->x == 0.f)
    std::cout << "x is zero\n";
  else
    std::cout << "x is positive\n";

  delete p;
  return 0;
}
```

Где здесь ошибка?

Компилируем без динамических проверок, запустим:

```sh
$ clang++-18 main.cpp -O3
$ ./a.out
x is positive
```

Программа "отработала".

Компилируем с инструментированием msan, запустим (приведённый сокращённый вывод):

```
$ clang++-18 main.cpp -O3 -fsanitize=memory
$ ./a.out
==10366==WARNING: MemorySanitizer: use-of-uninitialized-value
    #0 0x555555620302 in main
    #1 0x7ffff782a1c9 in __libc_start_call_main
    #2 0x7ffff782a28a in __libc_start_main
    #3 0x555555586304 in _start

SUMMARY: MemorySanitizer: use-of-uninitialized-value in main
Exiting
```

<br />

#### Как и когда следует пользоваться санитайзерами?

Для начала попробуйте ответить на этот вопрос самостоятельно.

Ответ (неполный):
  * Прогон инструментированных юнит-тестов
  * Прогон интеграционных тестов над инструментированной программой
  * Использование инструментированного продукта разработчиками / тестировщиками
  * Использование инструментированного продукта пользователями (*):
    * как решать негативные эффекты от санитайзеров - частичные решения в виде HWASAN
    * что делать при детекции ошибки - crash + crash report

<br />

### Фаззинг-тестирование

Идея примитивного фаззинг-тестирования - запускать функцию много-много раз на случайных данных.
  * Если можем посчитать корректный ответ иным способом - сверять ответы на каждый запуск.
  * Если нет иного способа получения ответа - проверяем, что приложение хотя бы не падает (*)

<br />

Иллюстрировть возможности фаззеров будем на примерах:

```c++
// main.cpp
#include <cassert>
#include <vector>

int function_1(int x1, int x2, int x3) {
  int rv = 0;
  if (x1 % 1000 == 7) {
    if (x2 % 100 == 7) {
      if (x3 % 100 == 7) {
        // Jackpot!
        assert(false);
      }
      rv += 1;
    }
    rv += 1;
  }
  return rv;
}

int function_2(int x1, int x2, int x3) {
  int rv = 0;
  if (x1 % 1000 == 7) {
    if (x2 % 100 == 7) {
      if (x3 % 100 == 7) {
        // Jackpot!
        std::vector<int> v = {1, 2, 3, 4};
        int &value = v[1];
        v.resize(1000);
        return value;
      }
      rv += 1;
    }
    rv += 1;
  }
  return rv;
}
```

Функции `function_1` и `function_2` с вероятностью 0.00001% попадают в ветку с ошибочным кодом. Такую ошибку легко пропустить при юнит-тестировании, а при достаточно большой пользовательской базе она начнёт "стрелять" у пользователя.

Фаззеры справляются с задачей поиска даже таких редких ошибок, если их корректно организовать.

<br />

#### Рукописный фаззинг

Самый примитивный вариант - рукописный фаззер, который будет случайным образом генерировать вход и запускать тестируемую функцию:



Пример рукописного раннера:

```c++
// manual_runner.cpp
#include <cassert>
#include <chrono>
#include <cstdlib>
#include <iostream>
#include <string>

int function_1(int x1, int x2, int x3);
int function_2(int x1, int x2, int x3);

int main(int argc, char **argv) {
  using namespace std::chrono;

  if (argc != 3) {
    std::cerr << "Usage: " << argv[0] << " [0|1] <num_seconds>\n";
    return 1;
  }

  const int func_id = std::stoi(argv[1]);
  const int num_seconds = std::stoi(argv[2]);

  const auto start_ts = high_resolution_clock::now();
  while (true) {
    const int x1 = std::rand();
    const int x2 = std::rand();
    const int x3 = std::rand();
    if (func_id == 0)
      function_1(x1, x2, x3);
    else
      function_2(x1, x2, x3);

    const auto final_ts = high_resolution_clock::now();
    if (duration_cast<seconds>(final_ts - start_ts).count() >= num_seconds)
      break;
  }

  return 0;
}
```

Компилируем под O3, запускаем потестировать `function_1` на минуту:

```
$ clang++-18 -O3 manual_runner.cpp main.cpp
$ time ./a.out 0 60
a.out: main.cpp:10: int function_1(int, int, int): Assertion `false' failed.
Aborted (core dumped)

real	0m0,534s
user	0m0,420s
sys	0m0,005s
```

Рукописный фаззер находит проблемный вход за полсекунды. Неплохо.

Если запустить на фаззинг `function_2` просто так, то фаззер, конечно, ничего не найдёт. Но давайте инструемнтируем код с помощью address sanitizer и запустим фаззер:

```
$ clang++-18 -O3 manual_runner.cpp main.cpp -fsanitize=address
$ time ./a.out 1 60
=================================================================
==17068==ERROR: AddressSanitizer: heap-use-after-free on address 0x502000000014 at pc 0x5ea6097d6743 bp 0x7ffd6f5bc640 sp 0x7ffd6f5bc638
READ of size 4 at 0x502000000014 thread T0
    #0 0x5ea6097d6742 in function_2(int, int, int)
    #1 0x5ea6097d61b1 in main
    #2 0x7b57bfc2a1c9 in __libc_start_call_main
    #3 0x7b57bfc2a28a in __libc_start_main
    #4 0x5ea6096fa3e4 in _start

0x502000000014 is located 4 bytes inside of 16-byte region [0x502000000010,0x502000000020)
freed by thread T0 here:
    #0 0x5ea6097d40d1 in operator delete(void*)
    #1 0x5ea6097d6718 in function_2(int, int, int)
    #2 0x7b57bfc2a1c9 in __libc_start_call_main
    #3 0x7b57bfc2a28a in __libc_start_main
    #4 0x5ea6096fa3e4 in _start

previously allocated by thread T0 here:
    #0 0x5ea6097d3851 in operator new(unsigned long)
    #1 0x5ea6097d670c in function_2(int, int, int)
    #2 0x7b57bfc2a1c9 in __libc_start_call_main
    #3 0x7b57bfc2a28a in __libc_start_main
    #4 0x5ea6096fa3e4 in _start

SUMMARY: AddressSanitizer: heap-use-after-free in function_2(int, int, int)
==17068==ABORTING

real	0m0,482s
user	0m0,417s
sys	0m0,008s
```

**Внимание:** Наш рукописный фаззер на коленке за те же 0.5 секунды нашёл сложный для поимки баг heap-use-after-free, который стреляет настолько редко, что никогда бы не поймался на юнит-тестах. Более того, он даже указал места аллокации, освобождения и использования памяти!

<br />

#### libfuzzer

https://llvm.org/docs/LibFuzzer.html

> libFuzzer – a library for coverage-guided fuzz testing.
>
> Some important things to remember about fuzz targets:
>   * The fuzzing engine will execute the fuzz target many times with different inputs in the same process.
>   * It must tolerate any kind of input (empty, huge, malformed, etc).
>   * It must not exit() on any input.
>   * It may use threads but ideally all threads should be joined at the end of the function.
>   * It must be as deterministic as possible. Non-determinism (e.g. random decisions not based on the input bytes) will make fuzzing inefficient.
>   * It must be fast. Try avoiding cubic or greater complexity, logging, or excessive memory consumption.
>   * Ideally, it should not modify any global state (although that’s not strict).
>   * Usually, the narrower the target the better. E.g. if your target can parse several data formats, split it into several targets, one per format.

Напомню, что мы хотим искать редкие ошибки в этих двух функциях:

```c++
int function_1(int x1, int x2, int x3); // contains assert(false)
int function_2(int x1, int x2, int x3); // contains ub
```

Чтобы использовать libfuzzer, программист должен реализовать прослойку, которая будет по случайному входу генерировать вызов тестируемой функции. От качества реализации этой прослойки зависит эффективность фаззера:

<br />

Реализуем прослойку фаззера для `function_1`:

```c++
// libfuzzer_runner_1.cpp
#include <cstdint>
#include <cstddef>

int function_1(int x1, int x2, int x3);

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *data, size_t size) {
  int x1 = 0;
  int x2 = 0;
  int x3 = 0;
  for (size_t i = 0; i < size; ++i) {
    switch (i % 3) {
    case 0: x1 = (x1 << 8) | data[i]; break;
    case 1: x2 = (x2 << 8) | data[i]; break;
    case 2: x3 = (x3 << 8) | data[i]; break;
    }
  }

  function_1(x1, x2, x3);
  return 0;
}
```

Заметьте, здесь нигде нет функции `main`!

Соберём и запустим (вывод редуцирован):

```
$ clang++-18 -O3 -g -fsanitize=fuzzer main.cpp libfuzzer_runner_1.cpp
a.out: main.cpp:10: int function_1(int, int, int): Assertion `false' failed.
==5734== ERROR: libFuzzer: deadly signal
    #0 0x5c928967d7c8 in __sanitizer_print_stack_trace
    #1 0x5c928965283c in fuzzer::PrintStackTrace()
    #2 0x5c92896388c7 in fuzzer::Fuzzer::CrashCallback()
    #3 0x75daf044531f
    #4 0x75daf049eb1b in __pthread_kill_implementation
    #5 0x75daf049eb1b in __pthread_kill_internal
    #6 0x75daf049eb1b in pthread_kill
    #7 0x75daf044526d in raise
    #8 0x75daf04288fe in abort
    #9 0x75daf042881a in __assert_fail_base
    #10 0x75daf043b506 in __assert_fail
    #11 0x5c928967ec07 in function_1(int, int, int) /home/ivafanas/projects/temp/main.cpp:10:9
    #12 0x5c928967ee7b in LLVMFuzzerTestOneInput /home/ivafanas/projects/temp/libfuzzer_runner_1.cpp:18:3
    #13 0x5c9289639e94 in fuzzer::Fuzzer::ExecuteCallback(unsigned char const*, unsigned long)
    #14 0x5c9289639589 in fuzzer::Fuzzer::RunOne(unsigned char const*, unsigned long, bool, fuzzer::InputInfo*, bool, bool*)
    #15 0x5c928963ad75 in fuzzer::Fuzzer::MutateAndTestOne()
    #16 0x5c928963b8d5 in fuzzer::Fuzzer::Loop(std::vector<fuzzer::SizedFile, std::allocator<fuzzer::SizedFile>>&)
    #17 0x5c9289628baf in fuzzer::FuzzerDriver(int*, char***, int (*)(unsigned char const*, unsigned long))
    #18 0x5c9289653236 in main
    #19 0x75daf042a1c9 in __libc_start_call_main
    #20 0x75daf042a28a in __libc_start_main
    #21 0x5c928961db94 in _start

SUMMARY: libFuzzer: deadly signal
0x0,0x0,0x7,0x7,0x7,
\000\000\007\007\007
artifact_prefix='./'; Test unit written to ./crash-904c66f541163b28f79147766ea263f167ef735a
Base64: AAAHBwc=

real	0m0,082s
user	0m0,009s
sys	0m0,018s
```

libfuzzer нашёл ошибку и справился с задачей в 6-7 раз быстрее нашего рукописного варианта!

<br />

Реализуем прослойку фаззера для `function_2`:

```c++
// libfuzzer_runner_2.cpp
#include <cstdint>
#include <cstddef>

int function_2(int x1, int x2, int x3);

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *data, size_t size) {
  int x1 = 0;
  int x2 = 0;
  int x3 = 0;
  for (size_t i = 0; i < size; ++i) {
    switch (i % 3) {
    case 0: x1 = (x1 << 8) | data[i]; break;
    case 1: x2 = (x2 << 8) | data[i]; break;
    case 2: x3 = (x3 << 8) | data[i]; break;
    }
  }

  function_2(x1, x2, x3);
  return 0;
}
```

Соберём и запустим (вывод редуцирован). **Обратите внимание на строку компиляции**:

```
$ clang++-18 -O3 -g -fsanitize=fuzzer,address libfuzzer_runner_2.cpp main.cpp
$ time ./a.out -seed=12345
==4032==ERROR: AddressSanitizer: heap-use-after-free on address 0x50200000ca74 at pc 0x5e0099b33b2d bp 0x7ffdfba5bb00 sp 0x7ffdfba5baf8
READ of size 4 at 0x50200000ca74 thread T0
    #0 0x5e0099b33b2c in function_2(int, int, int) /home/ivafanas/projects/temp/main.cpp:28:16
    #1 0x5e0099b33877 in LLVMFuzzerTestOneInput /home/ivafanas/projects/temp/libfuzzer_runner_2.cpp:18:3
    #2 0x5e0099a3ec64 in fuzzer::Fuzzer::ExecuteCallback(unsigned char const*, unsigned long)
    #3 0x5e0099a3e359 in fuzzer::Fuzzer::RunOne(unsigned char const*, unsigned long, bool, fuzzer::InputInfo*, bool, bool*)
    #4 0x5e0099a3fb45 in fuzzer::Fuzzer::MutateAndTestOne()
    #5 0x5e0099a406a5 in fuzzer::Fuzzer::Loop(std::vector<fuzzer::SizedFile, std::allocator<fuzzer::SizedFile>>&) 
    #6 0x5e0099a2d97f in fuzzer::FuzzerDriver(int*, char***, int (*)(unsigned char const*, unsigned long)) 
    #7 0x5e0099a58006 in main
    #8 0x77a3b142a1c9 in __libc_start_call_main
    #9 0x77a3b142a28a in __libc_start_main
    #10 0x5e0099a22964 in _start

0x50200000ca74 is located 4 bytes inside of 16-byte region [0x50200000ca70,0x50200000ca80)
freed by thread T0 here:
    #0 0x5e0099b31c11 in operator delete(void*)
    #1 0x5e0099b33ad5 in std::__new_allocator<int>::deallocate(int*, unsigned long)
    #2 0x5e0099b33ad5 in std::allocator_traits<std::allocator<int>>::deallocate(std::allocator<int>&, int*, unsigned long)
    #3 0x5e0099b33ad5 in std::_Vector_base<int, std::allocator<int>>::_M_deallocate(int*, unsigned long)
    #4 0x5e0099b33ad5 in std::vector<int, std::allocator<int>>::_M_default_append(unsigned long)
    #5 0x5e0099b33ad5 in std::vector<int, std::allocator<int>>::resize(unsigned long)
    #6 0x5e0099b33ad5 in function_2(int, int, int) /home/ivafanas/projects/temp/main.cpp:27:11
    #7 0x5e0099b33877 in LLVMFuzzerTestOneInput /home/ivafanas/projects/temp/libfuzzer_runner_2.cpp:18:3
    #8 0x5e0099a3ec64 in fuzzer::Fuzzer::ExecuteCallback(unsigned char const*, unsigned long)
    #9 0x5e0099a3e359 in fuzzer::Fuzzer::RunOne(unsigned char const*, unsigned long, bool, fuzzer::InputInfo*, bool, bool*)
    #10 0x5e0099a3fb45 in fuzzer::Fuzzer::MutateAndTestOne()
    #11 0x5e0099a406a5 in fuzzer::Fuzzer::Loop(std::vector<fuzzer::SizedFile, std::allocator<fuzzer::SizedFile>>&)
    #12 0x5e0099a2d97f in fuzzer::FuzzerDriver(int*, char***, int (*)(unsigned char const*, unsigned long))
    #13 0x5e0099a58006 in main
    #14 0x77a3b142a1c9 in __libc_start_call_main
    #15 0x77a3b142a28a in __libc_start_main
    #16 0x5e0099a22964 in _start

previously allocated by thread T0 here:
    #0 0x5e0099b31391 in operator new(unsigned long)
    #1 0x5e0099b33ac9 in std::__new_allocator<int>::allocate(unsigned long, void const*)
    #2 0x5e0099b33ac9 in std::allocator_traits<std::allocator<int>>::allocate(std::allocator<int>&, unsigned long)
    #3 0x5e0099b33ac9 in std::_Vector_base<int, std::allocator<int>>::_M_allocate(unsigned long)
    #4 0x5e0099b33ac9 in void std::vector<int, std::allocator<int>>::_M_range_initialize<int const*>(int const*, int const*, std::forward_iterator_tag)
    #5 0x5e0099b33ac9 in std::vector<int, std::allocator<int>>::vector(std::initializer_list<int>, std::allocator<int> const&)
    #6 0x5e0099b33ac9 in function_2(int, int, int) /home/ivafanas/projects/temp/main.cpp:25:30
    #7 0x5e0099b33877 in LLVMFuzzerTestOneInput /home/ivafanas/projects/temp/libfuzzer_runner_2.cpp:18:3
    #8 0x5e0099a3ec64 in fuzzer::Fuzzer::ExecuteCallback(unsigned char const*, unsigned long)
    #9 0x5e0099a3e359 in fuzzer::Fuzzer::RunOne(unsigned char const*, unsigned long, bool, fuzzer::InputInfo*, bool, bool*)
    #10 0x5e0099a3fb45 in fuzzer::Fuzzer::MutateAndTestOne()
    #11 0x5e0099a406a5 in fuzzer::Fuzzer::Loop(std::vector<fuzzer::SizedFile, std::allocator<fuzzer::SizedFile>>&)
    #12 0x5e0099a2d97f in fuzzer::FuzzerDriver(int*, char***, int (*)(unsigned char const*, unsigned long))
    #13 0x5e0099a58006 in main
    #14 0x77a3b142a1c9 in __libc_start_call_main
    #15 0x77a3b142a28a in __libc_start_main
    #16 0x5e0099a22964 in _start

SUMMARY: AddressSanitizer: heap-use-after-free /home/ivafanas/projects/temp/main.cpp:28:16 in function_2(int, int, int)
==4032==ABORTING
0x0,0x0,0x7,0x7,0x7,
\000\000\007\007\007
artifact_prefix='./'; Test unit written to ./crash-904c66f541163b28f79147766ea263f167ef735a
Base64: AAAHBwc=

real	0m0,109s
user	0m0,010s
sys	0m0,019s
```

И вновь libfuzzer справляется с поиском ошибки, причём в 5 раз быстрее рукописного перебора!

<br />

##### Важное упражение

Мы научились прогонять фаззеры, проверяющие программы на отсутствие падений и ошибок санитайзеров при исполнении кода функции. Как теперь написать фаззер для случая, когда мы умеем вычислять правильный ответ альтернативным методом?

<summary>
Ответ

<details>

```c++
// Функция, которую хотим протестировать с доп. проверкой
// на возвращаемый результат.
int my_function(int x);

// Функция, которую будем гонять под фаззингом.
void runner_for_my_function(int x) {
    int my_result = my_function(x);
    int ref_result = ref_function(x);
    assert(my_result == ref_result);
}
```
    
</details>
</summary>

<br />

##### AFL++

https://github.com/AFLplusplus/AFLplusplus

Мощный, богатый функционалом инструмент фаззинга:
1. В общем случае не требует реализации прослоек.
2. Может работать с чистыми бинарниками без исходного кода (но хуже и медленнее).
3. Может работать с gui-приложениями (но медленно).
4. Для эффективной работы требует пересборки исходников своим собственным компилятором.

Для самостоятельного изучения

<br />

#### Рекомендации по использованию фаззеров и санитайзеров

* Юнит-тесты гонять регулярно инструментированными санитайзерами.
* Интеграционный тесты гонять реруглярно над инструментированным санитайзером приложением.
* Использование инструментированного санитайзером продукта разработчиками / тестировщиками (хотя бы ubsan, лучше asan).
* Использовать разные санитайзеры, разные компиляторы, разные стандартные библиотеки (если позволяет продукт).
* Функциониал, который можно и осмысленно:
  * покрывать фаззинг-тестами
  * регулярно гонять фаззинг-тестирование в режиме минимум N минут
  * фаззинг выполнять над инструментированным санитайзером кодом.
* Качество тестирования определяется:
  * качеством покрытия кода юнит-/интеграционными тестами.
  * качеством покрытия сценариев при генерации в фаззинге.

**Из практики**: хорошо реализованный фаззинг находит весьма хитрые и неожиданные ошибки в коде, о существовании которых разработчик даже не подозревает.

<br />

**Упражнение:**

1. Реализуйте бинарный поиск по отсортированному массиву чисел. Массив может быть отсортирован как по возрастанию, так и по убыванию. Не забудьте, что для этого случая можно найти корректный ответ референсным алгоритмом. Напишите для него фаззер:
  * Через libfuzzer
  * Через ручной фаззинг.

Получилось сделать бин. поиск без ошибок с первого раза? :)

<br />

**Для самостоятельного изучения**:
* [Fuzzing: a survey. Jun Li, Bodong Zhao & Chao Zhang. Cybersecurity volume 1, Article number: 6 (2018)](https://cybersecurity.springeropen.com/articles/10.1186/s42400-018-0002-y)
* [LLVM libfuzzer doc](https://llvm.org/docs/LibFuzzer.html)
* [AFL best practices](https://github.com/AFLplusplus/AFLplusplus/blob/stable/docs/best_practices.md)
* [AFL tutorials](https://github.com/AFLplusplus/AFLplusplus/blob/stable/docs/tutorials.md)